In [ ]:
!pip install /120040051/downloaded_packages/torch-2.0.1+cu117-cp311-cp311-linux_x86_64.whl

In [ ]:
!pip install spacy
!pip install spacy-transformers

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
import spacy
from spacy.matcher import Matcher

In [ ]:
def extract_action_subjects(doc):
    named_entities = set(ent.text for ent in doc.ents)  # Exclude named entities.
    action_subjects = []
    for chunk in doc.noun_chunks:
        # Filter based on dependency tags, exclude personal pronouns, and ensure it's an action verb.
        if chunk.root.dep_ in ["nsubj", "nsubjpass"] and chunk.root.pos_ != "PRON":
            # Ensure the verb linked to the subject is an action verb
            head_verb = chunk.root.head
            if head_verb.pos_ == "VERB" and not any(ne in chunk.text or chunk.text in ne for ne in named_entities):
                subject = chunk.text
                action_subjects.append(subject)
    return action_subjects

In [ ]:
def extract_verbs_phrase(doc, nlp):
    verb_phrases = []
    matcher = Matcher(nlp.vocab)
    # Pattern for transitive and intransitive verb phrases
    transitive_pattern = [{"POS": "VERB"}, {"POS": "DET", "OP": "?"}, {"POS": "ADJ", "OP": "*"}, {"POS": "NOUN", "OP": "+"}]
    # intransitive_pattern = [{"POS": "VERB"}, {"POS": "ADP"}, {"POS": "DET", "OP": "?"}, {"POS": "ADJ", "OP": "*"}, {"POS": "NOUN", "OP": "+"}]

    # Add patterns to matcher
    matcher.add("verb_phrases", [transitive_pattern])
    # matcher.add("verb_phrases", [transitive_pattern, intransitive_pattern])

    matches = matcher(doc)
    for match_id, start, end in matches:
        span_text = doc[start:end].text.lower()
        found_subphrase = False 
        for i in range(len(verb_phrases)): # find if there is a sub-phrase in the list
            phrase = verb_phrases[i]
            if (phrase in span_text) and len(phrase) < len(span_text):
                verb_phrases[i] = span_text
                found_subphrase = True
            
        if not found_subphrase:
            verb_phrases.append(span_text)

    return verb_phrases

In [ ]:
# Remove the duplicates
def add_to_set(case_insensitive_set, items):
    for item in items:
        lower_item = item.lower()
        case_insensitive_set[lower_item] = case_insensitive_set.get(lower_item, 0) + 1

In [ ]:
from collections import Counter
def get_top_n_items(dictionary, n=150):
    counter = Counter(dictionary)
    return counter.most_common(n)

In [ ]:
# Load the transformer-based Spacy model
nlp = spacy.load("en_core_web_trf")

In [ ]:
idoc = nlp("The cat is a good animal.")
for chunk in idoc.noun_chunks:
    # Filter based on dependency tags and exclude personal pronouns.
    print(f"Chunk is [{chunk.text}] and Chunk's root is [{chunk.root.text}] and Chunk's root's head is [{chunk.root.head}]")
    # print(f"Chunk is {chunk.text} and Chunk's root is {chunk.root.text}")


In [ ]:
# Input file. The format should be one sentence per line.
with open("/120040051/test_resource/OMCS/omcs-sentences-more-en.txt", 'r') as file1:
    sentences = [line.strip() for line in file1.readlines()]

In [ ]:
total_len = len(sentences)
total_len

In [ ]:
# sentence_batch = sentences[:200]
# total_len = len(sentence_batch)
# len(sentence_batch)

In [ ]:
subjects = {}
verb_phrases = {}

i = 1
for sentence in sentences:
    doc = nlp(sentence)
    # Add items to the respective sets
    add_to_set(subjects, extract_action_subjects(doc))
    add_to_set(verb_phrases, extract_verbs_phrase(doc, nlp))
    i += 1

    if (i % 1000 == 0):
        print(f"Processed {i} / {total_len}")
print(f"Processed {i-1} / {total_len}")

In [ ]:
import json
with open('/120040051/test_resource/OMCS/OMCS-SUBJ-ALL.json', 'w') as f:
    json.dump(subjects, f)
with open('/120040051/test_resource/OMCS/OMCS-VERB-ALL.json', 'w') as f:
    json.dump(verb_phrases, f)

In [ ]:
# Filter to keep top 150 for each category
top_subjects = get_top_n_items(subjects, 500)
top_verb_phrases = get_top_n_items(verb_phrases, 500)

In [ ]:
with open('/120040051/test_resource/OMCS/OMCS-SUBJ-t500.txt', 'w') as file:
    for subject, freq in top_subjects:
        file.write(f"{subject}: {freq}\n")
print("Subjects writing complete.")

In [ ]:
with open('/120040051/test_resource/OMCS/OMCS-VERB-t500.txt', 'w') as file:
    for vp, freq in top_verb_phrases:
        file.write(f"{vp}: {freq}\n")
print("Verb phrases writing complete.")

In [ ]:
import requests
from collections import defaultdict

def get_related_verbs(nouns):
    base_url = "http://api.conceptnet.io/query"
    results = defaultdict(list)

    for noun in nouns:
        params = {
            "rel": "/r/CapableOf",
            "start": f"/c/en/{noun}",
            "limit": 1000  # Adjust limit as needed to fetch more results initially
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            edges = data.get("edges", [])
            for edge in edges:
                verb = edge.get("end", {}).get("label", "")
                weight = edge.get("weight", 0)
                results[noun].append((verb, weight))

            # Sort the verbs for each noun by weight in descending order and keep the top 10
            results[noun] = sorted(results[noun], key=lambda x: x[1], reverse=True)[:5]
        else:
            print(f"Failed to fetch data for {noun}")

    return results

In [ ]:
subject_wo_article = []
for subj in subjects:
    if subj.startswith(('a ', 'the ')):
        subject_wo_article.append(subj.split(' ')[1])
    else:
        subject_wo_article.append(subj)
subject_wo_article

In [ ]:
related_verbs_ = get_related_verbs(subject_wo_article)
for noun, verbs in related_verbs_.items():
    print(f"{noun.capitalize()} can:")
    for verb, weight in verbs:
        print(f"  - {verb} (Weight: {weight})")

In [ ]:
for noun, verbs in related_verbs_.items():
    # print(f"{noun.capitalize()} can:")
    for verb, weight in verbs:
        print(f"{verb}")

In [ ]:
# Open a new text file for writing
# file_path = Path("/mnt/data/filtered_verbs.txt")
filtered_verbs = set()
for noun, verbs in related_verbs_.items():
    for verb in verbs:
        sent = f"{noun.capitalize()} can {verb[0]}"
        doc = nlp(sent)
        filtered_verbs.add(extract_verbs_phrase(doc, nlp))
            # for phrase in filtered_verbs:

with open('/home/liu/test_prior/raw_results/omcs/filtered/verb_phrases_filtered_v2_add_5.txt', "w") as file:
    for verb in list(filtered_verbs):
        file.write(verb + "\n")